In [57]:
from math import floor, cos, acos, pi
import pyModeS as pms

def int2bin(val: int, bits: int) -> str:
    """Convert integer to binary string with left-zero padding to 'bits' length."""
    return f"{val:0{bits}b}"

def compute_NL(lat: float) -> int:
    """Compute NL (longitude zone number) function needed for CPR."""
        
    if abs(lat) < 10**-6: # When near the equator, NL is fixed
        return 59
    elif abs(lat) == 87: # Might be necessary to add some tolerance in the future
        return 2
    elif abs(lat) > 87: # Near the poles, NL is also fixed
        return 1
    else: # Computes the NL
        a = 1 - cos(pi / (2*NZ)) 
        b = cos(pi * lat / 180) ** 2
        nl = 2 * pi / (acos(1 - a / b))
        return int(floor(nl))

lat = 52.2657
lon = 3.91937

NZ = 15 # The number of latitude zones N_z
NL = compute_NL(lat)

dLat_even = 360.0 / (4 * NZ)
dLat_odd = 360.0 / (4 * NZ - 1)

dLon_even = 360.0 / max(NL, 1)
dLon_odd = 360.0 / max(NL-1, 1)

lat_idx_even = floor(lat/dLat_even)
lat_idx_odd = floor(lat/dLat_odd)

lon_idx_even = floor(lon/dLon_even)
lon_idx_odd = floor(lon/dLon_odd)

remainder_lat_even = lat - lat_idx_even * dLat_even
remainder_lat_odd = lat - lat_idx_odd * dLat_odd

remainder_lon_even = lon - lon_idx_even * dLon_even
remainder_lon_odd = lon - lon_idx_odd * dLon_odd

scaled_lat_even = remainder_lat_even/dLat_even
scaled_lat_odd = remainder_lat_odd/dLat_odd

scaled_lon_even = remainder_lon_even/dLon_even
scaled_lon_odd = remainder_lon_odd/dLon_odd

print(int(scaled_lat_even * 2**17), int(scaled_lat_odd * 2**17))
print(int(scaled_lon_even * 2**17), int(scaled_lon_odd * 2**17))

93185 74156
51371 49944


In [147]:
alt = 130031 + 1300
n500 = alt // 500
n100 = (alt % 500) // 100

print(n500)

# Step 2: Convert both to Gray code
gray_n500 = int2bin(int_to_gray(n500), 8)  # 8-bit
gray_n100 = int2bin(int_to_gray(n100), 3)  # 3-bit
graystr = gray_n500+gray_n100

print(graystr, len(graystr))

bitstring = ['0'] * 12

bitstring[0] = graystr[8]
bitstring[1] = graystr[2]
bitstring[2] = graystr[9]
bitstring[3] = graystr[3]
bitstring[4] = graystr[10]
bitstring[5] = graystr[4]
bitstring[6] = graystr[5]
bitstring[7] = '0' # Q bit
bitstring[8] = graystr[6]
bitstring[9] = graystr[0]
bitstring[10] = graystr[7]
bitstring[11] = graystr[1]

print(''.join(bitstring), len(''.join(bitstring)))

262
110000101010 12
100010001101 12


In [149]:
def gray2alt(binstr: str):
    gc500 = binstr[:8]
    n500 = gray2int(gc500)

    # in 100-ft step must be converted first
    gc100 = binstr[8:]
    n100 = gray2int(gc100)

    if n100 in [0, 5, 6]:
        return None

    if n100 == 7:
        n100 = 5

    if n500 % 2:
        n100 = 6 - n100

    alt = (n500 * 500 + n100 * 100) - 1300
    return alt

gray2alt('110000101010')

63600

In [156]:
pms.tell('8DC946C8499B866AF48CDECDB6CB')

                     Message: 8DC946C8499B866AF48CDECDB6CB 
                ICAO address: C946C8 
             Downlink Format: 17 
                    Protocol: Mode-S Extended Squitter (ADS-B) 
                        Type: Airborne position (with barometric altitude) 
                  CPR format: Odd 
                CPR Latitude: 0.6044464111328125 
               CPR Longitude: 0.2751312255859375 
                    Altitude: 30000 feet
